# GUARDANDO Y RECUPERANDO MODELOS CON KERAS

### Configuración previa

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

tf.keras.backend.clear_session()  # Para restablecer fácilmente el estado del portátil.

2024-04-16 16:17:33.011115: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 1. Guardando modelos secuenciales o modelos funcionales

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='3_layer_mlp')
model.summary()

Model: "3_layer_mlp"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 digits (InputLayer)         [(None, 784)]             0         
                                                                 
 dense_1 (Dense)             (None, 64)                50240     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 predictions (Dense)         (None, 10)                650       
                                                                 
Total params: 55050 (215.04 KB)
Trainable params: 55050 (215.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Se entrena el modelo, solo para que tenga valores de peso para guardarlos, así como un estado optimizador. Por supuesto, tambien se pueden guardar modelos que nunca se han entrenado, pero obviamente eso es menos interesante.

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop())
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1)

938/938 [==============================] - 2s 2ms/step - loss: 0.3155


In [5]:
# Guardar predicciones para futuras verificaciones
predictions = model.predict(x_test)

313/313 [==============================] - 0s 1ms/step


#### Modelo-Completo Guardando
Se puede guardar un modelo creado con la API funcional en un solo archivo. Posteriormente, se puede volver a crear el mismo modelo a partir de este archivo, incluso si ya no tiene acceso al codigo que creó el modelo.

Este archivo incluye:

- Los modelos de arquitectura
- Los valores de peso del modelo (que se aprendieron durante el entrenamiento)
- La configuración de entrenamiento del modelo (lo que pasó a 'compilar'), si corresponde
- El optimizador y su estado, si corresponde (esto le permite reiniciar el entrenamiento donde lo dejó)

In [6]:
# Guardar el Modelo
model.save('path_to_my_model.h5')

# Recrea exactamente el mismo modelo solo desde el archivo
new_model = keras.models.load_model('path_to_my_model.h5')

/Users/jordi/anaconda3/envs/env_ceia/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
import numpy as np

# Verificar que el estado esté preservado
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Es necesario tener en cuenta que el estado del optimizador también se conserva:
# se puede reanudar el entrenamiento donde se dejó.

313/313 [==============================] - 0s 1ms/step


### Solo arquitectura de guardado   

A veces, solo interesa guardar la arquitectura del modelo y no se necesita guardar los valores de peso o el optimizador. En este caso, se puede recuperar la "configuracion" del modelo mediante el metodo get_config (). La configuracion es un dict de Python que permite recrear el mismo modelo, inicializado desde cero, sin ninguna de la información aprendida previamente durante el entrenamiento.

In [10]:
config = model.get_config()
reinitialized_model = keras.Model.from_config(config)

# ¡Se debe tener en cuenta que el estado del modelo no se conserva! Solo guardamos la arquitectura.
new_predictions = reinitialized_model.predict(x_test)
assert abs(np.sum(predictions - new_predictions)) > 0.

313/313 [==============================] - 0s 1ms/step


Alternativamente, se puede usar 'to_json()' de 'from_json ()', que usa una cadena JSON para almacenar la configuracion en lugar de un 'dict' de Python. Esto es util para guardar la configuracion en el disco.

In [11]:
json_config = model.to_json()
reinitialized_model = keras.models.model_from_json(json_config)

### Guardando solo con pesos
A veces, solo interesa el estado del modelo, sus valores de peso, y no la arquitectura. En este caso, se puede recuperar los valores de pesos como una lista de matrices Numpy a traves de 'get_weights()', y establecer el estado del modelo a través de 'set_weights'

In [12]:
weights = model.get_weights()  # Recupera el estado del modelo.
model.set_weights(weights)  # Establece el estado del modelo.

Se puede combinar **get_config()/from_config()** y **get_weights()/set_weights()** para recrear el modelo en el mismo estado. Sin embargo, a diferencia de **model.save()**, esto no incluirá la configuración de entrenamiento y el optimizado tendría que volver a llamar a **compile()** antes de usar el modelo para el entrenamiento.

In [13]:
config = model.get_config()
weights = model.get_weights()

new_model = keras.Model.from_config(config)
new_model.set_weights(weights)

# Verificar que el estado esté preservado
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Se debe tener en cuenta que el optimizador no se conserva
# entonces el modelo debe compilarse nuevamente antes de entrenar
# (y el optimizador comenzará desde un estado en blanco).

313/313 [==============================] - 0s 1ms/step


La alternativa de guardar en disco a 'get_weights()' y 'set_weights(weights)' es 'save_weights(fpath)' y 'load_weights(fpath)'.

A continuación se muestra un ejemplo que guarda en el disco:

In [14]:
# Guardar configuración JSON en el disco
json_config = model.to_json()
with open('model_config.json', 'w') as json_file:
    json_file.write(json_config)
# Guardar pesos en el disco
model.save_weights('path_to_my_weights.h5')

# Recargar el modelo de los 2 archivos que guardamos
with open('model_config.json') as json_file:
    json_config = json_file.read()
new_model = keras.models.model_from_json(json_config)
new_model.load_weights('path_to_my_weights.h5')

# Verificar que el estado esté preservado
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Se debe tener en cuenta que el optimizador no se conserva.

313/313 [==============================] - 0s 1ms/step


## IMPORTANTE

De todo lo visto en este notebook, la forma más sencilla, más usada y más recomendada es la siguiente:

In [15]:
# LEGACY
model.save('path_to_my_model.h5')
del model
model = keras.models.load_model('path_to_my_model.h5')

/Users/jordi/anaconda3/envs/env_ceia/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
# ACTUAL FORMAT
model.save('path_to_my_model.keras')
del model
model = keras.models.load_model('path_to_my_model.keras')